# <span style='font-family:"Times New Roman"'> <span styel=''>**CBIOPORTAL EXTRACTION OF MSK IMPACT MUTATIONS**
## <span style='font-family:"Times New Roman"'> <span styel=''>*Emile Cohen*
 *March 2020*

**Goal:** Through this notebook, we call the CBioPortal API to get the mutations file of the entire MSK IMPACT Cohort.

---

# Creation of exploitable dataframe from CBioPortal Datasets

In [4]:
%run -i '../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')

data_path = '../../data/'

import pprint
import json
import click
import requests

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

## Extract dataset from Cbio Portal API
First we need to extract the samples from the Clinical Data Table that we have already downloaded from CBioPortal

In [5]:
samples = pd.read_csv(data_path + 'cbioportal/raw/mskimpact_clinical_data-2.tsv', sep= '\t')
samples

,Study ID,Patient ID,Sample ID,Diagnosis Age,Age at Which Sequencing was Reported (Days),Patient Current Age,Archer Panel,Cancer Type,Cancer Type Detailed,CRDB_ADJ_TXT,CRDB_BASIC_COMMENTS,CRDB_BRAINMET,CRDB_CONSENT_DATE_DAYS,CRDB_ECOG,CRDB_NOSYSTXT,CRDB_OFF_STUDY_DAYS,CRDB_PRIOR_RX,CRDB_SURVEY_COMMENTS,CRDB_SURVIVAL_STATUS,CRDB_TREATMENT_END_DAYS,Impact TMB Percentile (Across All Tumor Types),Impact TMB Score,Impact TMB Percentile (Within Tumor Type),Date added to cBioPortal,Disease Free (Months),Disease Free Status,Ethnicity Category,Fraction Genome Altered,Gene Panel,Neoplasm Histologic Type Name,Institute Source,Metastatic Site,MGMT Status,Month added to cBioPortal,MSI Comment,MSI Score,MSI Type,MSK Slide ID,Mutation Count,Oncotree Code,Overall Survival (Months),Overall Survival Months Reported by DMT,Overall Survival Status,Overall Status Reported by DMT,Other Patient ID,12-245 Part A Consented,12-245 Part C Consented,MSK Pathology Slide Available,Pediatric Case Indicator,Primary Tumor Site,Race Category,Religion,Sample Class,Number of Samples Per Patient,Sample coverage,Sample Type,Sex,Sex Reported by DMT,Somatic Status,SO comments,Tumor Purity,Week added to cBioPortal,WHO Grade
0,mskimpact,P-0000004,P-0000004-T01-IM3,NaN,40,40.0,NO,Breast Cancer,Breast Invasive Ductal Carcinoma,YES,NaN,NO,14484.0,0.0,6.0,14631.0,YES,NaN,Dead,14631.0,58.6,4.5,67.8,2015/04/07,NaN,NaN,Non-Spanish; Non-Hispanic,0.2782,IMPACT341,NaN,MSKCC,NaN,NaN,2015/04,NaN,2.50,Stable,NaN,4,IDC,3.551,NaN,DECEASED,NaN,DMP0004,YES,NO,NO,No,Breast,WHITE,NONE,Tumor,1,428,Primary,Female,NaN,Matched,NaN,50,"2015, Wk. 15",NaN
1,mskimpact,P-0000012,P-0000012-T02-IM3,NaN,59,64.0,NO,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,NaN,NaN,21192.0,NaN,NaN,NaN,NaN,NaN,Alive,NaN,17.1,1.1,13.5,2015/04/07,NaN,NaN,Non-Spanish; Non-Hispanic,0.3146,IMPACT341,NaN,MSKCC,NaN,NaN,2015/04,MICROSATELLITE INSTABILITY-INDETERMINATE. See MSI note below.,4.10,Indeterminate,NaN,1,IDC,72.460,NaN,LIVING,NaN,NaN,YES,NO,NO,No,Breast,WHITE,NONE,Tumor,3,344,Primary,Female,NaN,Matched,NaN,NaN,"2015, Wk. 15",NaN
2,mskimpact,P-0000012,P-0000012-T03-IM3,NaN,59,64.0,NO,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,NaN,NaN,NaN,21192.0,NaN,NaN,NaN,NaN,NaN,Alive,NaN,96.7,33.5,98.3,2015/04/07,NaN,NaN,Non-Spanish; Non-Hispanic,0.1844,IMPACT341,NaN,MSKCC,Neck,NaN,2015/04,MICROSATELLITE STABLE (MSS). See MSI note below.,0.47,Stable,NaN,30,LUAD,72.460,NaN,LIVING,NaN,NaN,YES,NO,NO,No,Lung,WHITE,NONE,Tumor,3,428,Metastasis,Female,NaN,Matched,NaN,NaN,"2015, Wk. 15",NaN
3,mskimpact,P-0000012,P-0000012-T04-IM6,NaN,64,64.0,YES,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,NaN,NaN,NaN,21192.0,NaN,NaN,NaN,NaN,NaN,Alive,NaN,79.1,7.9,63.8,2018/08/01,NaN,NaN,Non-Spanish; Non-Hispanic,0.2250,IMPACT468,NaN,MSKCC,NaN,NaN,2018/08,MICROSATELLITE STABLE (MSS). See MSI note below.,0.20,Stable,1103618.0,9,LUAD,72.460,NaN,LIVING,NaN,NaN,YES,NO,YES,No,Lung,WHITE,NONE,Tumor,3,713,Primary,Female,NaN,Matched,Note: The mutations and copy number profile suggests that this sample may be clonally unrelated to the prior sample M13-9122. Note: Allele specific copy number analysis by FACETS suggests copy neutral loss of heterozygosity for BRCA1. FACETS results are f,20,"2018, Wk. 31",NaN
4,mskimpact,P-0000015,P-0000015-T01-IM3,NaN,45,45.0,NO,Breast Cancer,Breast Invasive Ductal Carcinoma,NO,NaN,NO,16221.0,1.0,6.0,16656.0,NO,NaN,Dead,16656.0,78.6,7.8,88.2,2015/04/07,NaN,NaN,Non-Spanish; Non-Hispanic,0.3503,IMPACT341,NaN,MSKCC,Liver,NaN,2015/04,NaN,2.55,Stable,NaN,7,IDC,13.677,NaN,DECEASED,NaN,DMP0003,YES,NO,NO,No,Breast,WHITE,NONE,Tumor,1,281,Metastasis,Female,NaN,Matched,NaN,40,"2015, Wk. 15",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52965,mskimpact,P-0053034,P-0053034-T01-IM6,NaN,NaN,NaN,NO,Endometrial Cancer,Uterin

In [15]:
%%time

def retry_request(method, **kwargs):
    """Retry request operation multiple times."""
    url = kwargs.get('url') or ""
    click.secho(f"Querying to {url}", fg="green")
    for i in [0.2, 1, 5, 10, 60, 90, 120, 300]:  # attempt some retries
        try:
            error = None
            response = getattr(requests, method)(verify=False, **kwargs)
        except requests.exceptions.RequestException as request_error:  # pragma: no cover
            error = request_error
            response = None
        if response is not None and not str(response.status_code).startswith("50"):
            break
        else:  # pragma: no cover
            msg = f"Request to {url} failed with error: {error}, retrying in {i}s..."
            click.secho(msg, fg="yellow", err=True)
            time.sleep(i)
    return response

endpoints =  { 
    'samples': 'https://cbioportal.mskcc.org/api/studies/mskimpact/patients/{pid}/samples?projection=DETAILED',
    'mutations': 'https://cbioportal.mskcc.org/api/molecular-profiles/mskimpact_mutations/mutations/fetch?projection=DETAILED',
    'segments': 'https://cbioportal.mskcc.org/api/copy-number-segments/fetch?projection=DETAILED'
}

headers = {
    "Authorization": f"Bearer 82e01e74-dd2a-412a-9a19-fda2e5d5d519",
    "Content-Type": "application/json",
}

if samples.any().any():
    sampleIds = list(samples['Sample ID'])[0:2]
    #click.secho(f'Patient {pid} has samples: {", ".join(sampleIds)}')
    # Get Mutations for all DMP Samples
    data = {'sampleIds': sampleIds}
    response = retry_request(
        'post',
        url=endpoints['mutations'], 
        data=json.dumps(data),
        headers=headers
    )
    mutations = [{**d, **d["gene"]} for d in json.loads(response.content)] 

Querying to https://cbioportal.mskcc.org/api/molecular-profiles/mskimpact_mutations/mutations/fetch?projection=DETAILED
CPU times: user 322 ms, sys: 76.5 ms, total: 399 ms
Wall time: 723 ms


In [20]:
mut = pd.DataFrame(mutations)
#get_groupby(mut, 'mutationStatus', 'count')


,uniqueSampleKey,uniquePatientKey,molecularProfileId,sampleId,patientId,entrezGeneId,gene,studyId,center,mutationStatus,validationStatus,tumorAltCount,tumorRefCount,normalAltCount,normalRefCount,startPosition,endPosition,referenceAllele,proteinChange,mutationType,functionalImpactScore,fisValue,linkXvar,linkPdb,linkMsa,ncbiBuild,variantType,keyword,driverFilter,driverFilterAnnotation,driverTiersFilter,driverTiersFilterAnnotation,chr,variantAllele,refseqMrnaId,proteinPosStart,proteinPosEnd,hugoGeneSymbol,type
0,UC0wMDAwMDA0LVQwMS1JTTM6bXNraW1wYWN0,UC0wMDAwMDA0Om1za2ltcGFjdA,mskimpact_mutations,P-0000004-T01-IM3,P-0000004,207,"{'entrezGeneId': 207, 'hugoGeneSymbol': 'AKT1', 'type': 'protein-coding'}",mskimpact,MSKCC,SOMATIC,Unknown,244,202,1,711,105246551,105246551,C,E17K,Missense_Mutation,M,2.175000e+00,"getma.org/?cm=var&var=hg19,14,105246551,C,T&fts=all",getma.org/pdb.php?prot=AKT1_HUMAN&from=6&to=108&var=E17K,getma.org/?cm=msa&ty=f&p=AKT1_HUMAN&rb=6&re=108&var=E17K,GRCh37,SNP,AKT1 E17 missense,,,,,14,T,NA,17,17,AKT1,protein-coding
1,UC0wMDAwMDA0LVQwMS1JTTM6bXNraW1wYWN0,UC0wMDAwMDA0Om1za2ltcGFjdA,mskimpact_mutations,P-0000004-T01-IM3,P-0000004,7157,"{'entrezGeneId': 7157, 'hugoGeneSymbol': 'TP53', 'type': 'protein-coding'}",mskimpact,MSKCC,SOMATIC,Unknown,58,209,0,600,7578503,7578518,CAGGGCAGGTCTTGGC,A138Cfs*27,Frame_Shift_Del,NA,1.401300e-45,NA,NA,NA,GRCh37,DEL,TP53 truncating,,,,,17,-,"NM_001126112.2,NM_001276761.1,NM_001276760.1,NM_000546.5,NM_0011",138,143,TP53,protein-coding
2,UC0wMDAwMDEyLVQwMi1JTTM6bXNraW1wYWN0,UC0wMDAwMDEyOm1za2ltcGFjdA,mskimpact_mutations,P-0000012-T02-IM3,P-0000012,7157,"{'entrezGeneId': 7157, 'hugoGeneSymbol': 'TP53', 'type': 'protein-coding'}",mskimpact,MSKCC,SOMATIC,Unknown,114,113,0,569,7577515,7577515,T,T256P,Missense_Mutation,M,3.140000e+00,"getma.org/?cm=var&var=hg19,17,7577515,T,G&fts=all",getma.org/pdb.php?prot=P53_HUMAN&from=95&to=289&var=T256P,getma.org/?cm=msa&ty=f&p=P53_HUMAN&rb=95&re=289&var=T256P,GRCh37,SNP,TP53 T256 missense,,,,,17,G,"NM_001126112.2,NM_001276761.1,NM_001276760.1,NM_000546.5,NM_0011",256,256,TP53,protein-coding
3,UC0wMDAwMDA0LVQwMS1JTTM6bXNraW1wYWN0,UC0wMDAwMDA0Om1za2ltcGFjdA,mskimpact_mutations,P-0000004-T01-IM3,P-0000004,23013,"{'entrezGeneId': 23013, 'hugoGeneSymbol': 'SPEN', 'type': 'protein-coding'}",mskimpact,MSKCC,SOMATIC,Unknown,73,400,0,1071,16265908,16265908,A,I3661F,Missense_Mutation,M,2.275000e+00,"getma.org/?cm=var&var=hg19,1,16265908,A,T&fts=all",getma.org/pdb.php?prot=MINT_HUMAN&from=3498&to=3664&var=I3661F,getma.org/?cm=msa&ty=f&p=MINT_HUMAN&rb=3498&re=3664&var=I3661F,GRCh37,SNP,SPEN I3661 missense,,,,,1,T,NM_015001.2,3661,3661,SPEN,protein-coding
4,UC0wMDAwMDA0LVQwMS1JTTM6bXNraW1wYWN0,UC0wMDAwMDA0Om1za2ltcGFjdA,mskimpact_mutations,P-0000004-T01-IM3,P-0000004,58508,"{'entrezGeneId': 58508, 'hugoGeneSymbol': 'KMT2C', 'type': 'protein-coding'}",mskimpact,MSKCC,SOMATIC,Unknown,11,84,0,193,151945083,151945083,C,M812I,Missense_Mutation,L,8.050000e-01,"getma.org/?cm=var&var=hg19,7,151945083,C,T&fts=all",NA,getma.org/?cm=msa&ty=f&p=MLL3_HUMAN&rb=639&re=838&var=M812I,GRCh37,SNP,KMT2C M812 missense,,,,,7,T,NM_170606.2,812,812,KMT2C,protein-coding


In [ ]:
mut_table.to_pickle(data_path + 'cbioportal/raw/mutations_cohort.pkl')

In [23]:
mut_table = pd.read_pickle(data_path + 'cbioportal/raw/mutations_cohort.pkl')
master = pd.read_pickle(data_path + 'merged_data/master_file.pkl')

In [35]:
master_tp53 = master[master['tp53_count'] >=1]

In [41]:
samples = list(master['Tumor_Id'])

mut_table_filtered = mut_table[mut_table['sampleId'].isin(samples)]

mut_table_filtered_tp53 = mut_table_filtered[mut_table_filtered['hugoGeneSymbol']== 'TP53']
get_groupby(mut_table_filtered_tp53, 'mutationStatus', 'count')

,count
mutationStatus,
GERMLINE,26
NA,136
SOMATIC,13984
UNKNOWN,83


In [46]:
list(mut_table_filtered_tp53[mut_table_filtered_tp53['mutationStatus']== 'GERMLINE']['sampleId'])

['P-0001586-T02-IM5',
 'P-0009076-T01-IM5',
 'P-0009076-T04-IM6',
 'P-0009076-T05-IM6',
 'P-0009076-T06-IM6',
 'P-0011220-T01-IM5',
 'P-0012425-T01-IM5',
 'P-0014507-T01-IM6',
 'P-0014726-T01-IM6',
 'P-0016090-T01-IM6',
 'P-0017801-T01-IM6',
 'P-0017801-T02-IM6',
 'P-0019349-T01-IM6',
 'P-0021028-T01-IM6',
 'P-0031705-T01-IM6',
 'P-0032271-T01-IM6',
 'P-0034664-T01-IM6',
 'P-0034797-T02-IM6',
 'P-0035720-T01-IM6',
 'P-0037899-T01-IM6',
 'P-0040092-T01-IM6',
 'P-0040825-T01-IM6',
 'P-0041242-T01-IM6',
 'P-0043786-T01-IM6',
 'P-0043787-T01-IM6',
 'P-0046911-T01-IM6']

In [38]:
len(samples)

12731

In [39]:
get_groupby(master, 'wgd', 'count')

,count
wgd,
-1.0,17895
1.0,9272


In [47]:
germ_samples = ['P-0001586-T02-IM5',
 'P-0009076-T01-IM5',
 'P-0009076-T04-IM6',
 'P-0009076-T05-IM6',
 'P-0009076-T06-IM6',
 'P-0011220-T01-IM5',
 'P-0012425-T01-IM5',
 'P-0014507-T01-IM6',
 'P-0014726-T01-IM6',
 'P-0016090-T01-IM6',
 'P-0017801-T01-IM6',
 'P-0017801-T02-IM6',
 'P-0019349-T01-IM6',
 'P-0021028-T01-IM6',
 'P-0031705-T01-IM6',
 'P-0032271-T01-IM6',
 'P-0034664-T01-IM6',
 'P-0034797-T02-IM6',
 'P-0035720-T01-IM6',
 'P-0037899-T01-IM6',
 'P-0040092-T01-IM6',
 'P-0040825-T01-IM6',
 'P-0041242-T01-IM6',
 'P-0043786-T01-IM6',
 'P-0043787-T01-IM6',
 'P-0046911-T01-IM6']

get_groupby(master[master['Tumor_Id'].isin(germ_samples)], 'Cancer_Type', 'count')

,count
Cancer_Type,
Bladder Cancer,1
Bone Cancer,1
Breast Cancer,3
Cancer of Unknown Primary,1
Choroid Plexus Tumor,1
Embryonal Tumor,1
Esophagogastric Cancer,1
Glioma,3
Melanoma,1
